In [ ]:
isinstance(obj, cls) # 检查 obj 是否是类 cls 的对象
issubclass(sub, super) # sub 是否是 super 类的派生类

In [6]:
# 反射：利用 字符串 操作DURC 对象/Module 成员，是一种基于字符串的事件驱动！
'''
Python中一切皆是对象（包括类也是），对象及其属性都是以一种字符串的形式展现的，于是：
    反射，通过字符串的形式操作对象及其属性，
    在字符串和对象之间建立一种映射关系，当程序作用于该字符串时，Python解释器可以知道，操作对应的对象


四个自省函数：类也是对象，以下 obj -> cls ，同样成立
    hasattr(obj, name) # 判断 object 是否有 name 方法或者属性
    getattr(obj, name [, default=None]) # 获取obj.name ，如果不存在，返回定义的default，如果没有定义default，则raise Exception
    setattr(obj, name, value) # obj.name = value，并更新 obj.__dict__
    delattr(obj, name) # 删除对象属性 del obj.name， 并更新 obj.__dict__


为什么用反射：

    1.实现可拔插机制：可以先定义好接口，后期真正需要实现的时候，再具体实现，不影响其他模块
    
    2.动态导入模块（基于反射当前模块成员）
        importlib.import_module('import_lib.metaclass')
    
'''

# _*_ EG1. 反射当前模块成员 _*_ #
import sys

def s1():
    pass
def s2():
    pass

print(hasattr(sys.modules[__name__], 's1'))
print(hasattr(sys.modules[__name__], 's2'))

print(sys.modules[__name__])

# 导入其他模块，可以利用反射查找该模块是否存在某个方法
# import example_module
# hasattr(exmple_module, attribution)

True
True
<module '__main__'>


In [9]:
# EG2. __setattr__, __getattr__, __delattr__ 用法 #
class Foo:
    def __init__(self, x):
        self.x = x
    
    def __getattr__(self, item):
        print('From getattr: 你找的属性不存在')
        
    def __setattr__(self, key, value):
        # self.key = value 这种方式会导致无限递归，因为 setattr 就是用 __setattr__ 去更新self.__dict__
        self.__dict__[key] = value
    
    def __delattr__(self, item):
        # del self.item 无限递归
        self.__dict__.pop(item)
        
f1 = Foo(10) # insert/update 触发 self.__setattr__
f1.z = 3
f1.__dict__['a'] = 3
print(f1.__dict__)

del f1.a # 触发 self.__delattr__
print(f1.__dict__)

# __getattr__ 只有在使用点调用属性，且属性不存在的时候才会触发
print(f1.x)
f1.xxxx

{'x': 10, 'z': 3, 'a': 3}
{'x': 10, 'z': 3}
10
From getattr: 你找的属性不存在


In [12]:
# _*_ EG3. 二次加工标准类（包装），实现授权 _*_ #
# 授权的过程，即是所有更新的功能都是由新类的某部分来处理，但已存在的功能就授权给对象的默认属性

class List(list):
    def __init__(self, item, tag=False):
        super().__init__(item)
        self.tag = tag
        
    def append(self, item):
        if not isinstance(item, str):
            raise TypeError
        super().append(item)
        
    def clear(self):
        if not self.tag:
            raise PermissionDeny
        super().clear()
        
l = List([1,2,3,4],False)
l.append('saf')
print(l, l.tag)

# l.clear() PermissionDeny

l.tag = True
l.clear()
print(l)

[1, 2, 3, 4, 'saf'] False
[]


In [2]:
# _*_ EG4. 覆盖 __getattr__ ,实现授权 _*_ #

# 示范一
class FileHandle:
    def __init__(self, filename, mode='r', encoding='utf-8'):
        if 'b' in mode:
            self.file = open(filename, mode)
        else:
            self.file = open(filename, mode, encoding=encoding)
        self.filename, self.mode, self.encoding = filename, mode, encoding
        
    def write(self, item):
        if 'b' in self.mode:
            if not isinstance(item, bytes):
                raise TypeError
        self.file.write(item)
        
    def __getattr__(self, item):
        print('属性不存在时，你才能看到我')
        return getattr(self.file, item)
    
    def __str__(self):
        if 'b' in self.mode:
            res = "<_io.BufferedReader name='%s' >" %self.filename
        else:
            res = "<_io.TextIOWrapper name='%s' mode='%s' encoding='%s'>" %(self.filename, self.mode, self.encoding)
        return res

f = FileHandle('a.txt','w+')
f.write('你好，小3')
f.seek(0)
print(f.read())
f.close()

f1 = FileHandle('a.txt','wb')
f1.write('你好，小1'.encode('utf-8')) # bytes 方式写入
f1.write('你好，小2'.encode('utf-8'))
f1.seek(0)
#print(f1.read()) wb 模式不可读，w+ 模式可读
print(f1)
f1.close()

属性不存在时，你才能看到我
属性不存在时，你才能看到我
你好，小3
属性不存在时，你才能看到我
属性不存在时，你才能看到我
<_io.BufferedReader name='a.txt' >
属性不存在时，你才能看到我


In [3]:
# _*_ EG5. __getattribute__ _*_ #
class Foo:
    def __init__(self,x):
        self.x = x
    
    def __getattr__(self, item):
        print('找不到属性时，我才会执行')
        # return self.__dict__ 无限递归
    
    def __getattribute__(self, item):
        print('不管属性是否存在，我都会执行')
        raise AttributeError('AH')
        
f = Foo(10)
f.x
f.xxx

'''
当__getattribute__与__getattr__同时存在,只会执行__getattrbute__,
除非__getattribute__在执行过程中抛出异常AttributeError
比如此处执行 f.x 时，__getattribute__ 抛出异常，然后 __getattr__ 继续执行
'''

不管属性是否存在，我都会执行


'\n当__getattribute__与__getattr__同时存在,只会执行__getattrbute__,\n除非__getattribute__在执行过程中抛出异常AttributeError\n'

In [12]:
# _*_ 描述符1 _*_ #
'''
描述符的本质是一个新式类，在新式类中，至少实现了 __get__, __set__, __delete__ 中的一个
亦称：描述符协议

描述符的作用是用来代理另外一个类的属性的 （描述符必须定义为类属性，不能是构造函数）
因为是类属性，故其实例化对象进行属性的调用/赋值/删除时，不会触发描述符的这三个方法
'''
# 描述符 Str
class Str:
    def __get__(self, instance, owner):
        print('Str调用')
        
    def __set__(self, instance, value):
        print('Str设置')
        
    def __delete__(self, instance):
        print('Str删除')

# 描述符 Int
class Int:
    def __get__(self, instance, owner):
        print('Int调用')
        
    def __set__(self, instance, value):
        print('Int设置')
        
    def __delete__(self, instance):
        print('Int删除')

class People:
    # 将变量定义为另外一个类的类属性：name被Str类代理,age被Int类代理
    name = Str()
    age = Int()
    def __init__(self, name, age):
        self.name, self.age = name, age

p = People('alex', 20)

# 描述符的使用
''' 描述符对类没有作用
People.name
People.name = 'egon'
del People.name
'''
p.name = 'egon'
p.name
del p.name
 
p.age
p.age = 19
del p.age

# 发生了什么
print(p.__dict__) # 实例对象并没有继承到任何属性
print(People.__dict__)
print(type(p) == People) # type(obj)其实是查看obj是由哪个类实例化来的
print(type(p).__dict__ == People.__dict__)

Str设置
Int设置
Str设置
Str调用
Str删除
Int调用
Int设置
Int删除
{}
{'__module__': '__main__', 'name': <__main__.Str object at 0x000001B310FA8C10>, 'age': <__main__.Int object at 0x000001B310FA8BB0>, '__init__': <function People.__init__ at 0x000001B31146DAF0>, '__dict__': <attribute '__dict__' of 'People' objects>, '__weakref__': <attribute '__weakref__' of 'People' objects>, '__doc__': None}
True
True


In [ ]:
# _*_ 描述符2 _*_ #
'''
如果每一个类属性都用这种方式实现，就太麻烦了，这时候装饰器就大显身手了
'''
# 无参
def decorate(cls):
    print('类的装饰器开始运行啦------>')
    return cls
 
@decorate #无参:People=decorate(People)
class People:
    def __init__(self,name,age,salary):
        self.name=name
        self.age=age
        self.salary=salary
 
p1=People('egon',18,3333.3)

# 有参
def typeassert(**kwargs):
    def decorate(cls):
        print('类的装饰器开始运行啦------>',kwargs)
        return cls
    return decorate

@typeassert(name=str,age=int,salary=float) #有参:1.运行typeassert(...)返回结果是decorate,此时参数都传给kwargs 2.People=decorate(People)
class People:
    def __init__(self,name,age,salary):
        self.name=name
        self.age=age
        self.salary=salary
 
p1=People('egon',18,3333.3)

# 万能型
class Typed:
    def __init__(self,name,expected_type):
        self.name=name
        self.expected_type=expected_type
    def __get__(self, instance, owner):
        print('get--->',instance,owner)
        if instance is None:
            return self
        return instance.__dict__[self.name]
 
    def __set__(self, instance, value):
        print('set--->',instance,value)
        if not isinstance(value,self.expected_type):
            raise TypeError('Expected %s' %str(self.expected_type))
        instance.__dict__[self.name]=value
    def __delete__(self, instance):
        print('delete--->',instance)
        instance.__dict__.pop(self.name)
 
def typeassert(**kwargs):
    def decorate(cls):
        print('类的装饰器开始运行啦------>',kwargs)
        for name,expected_type in kwargs.items():
            setattr(cls,name,Typed(name,expected_type))
        return cls
    return decorate
@typeassert(name=str,age=int,salary=float) #有参:1.运行typeassert(...)返回结果是decorate,此时参数都传给kwargs 2.People=decorate(People)
class People:
    def __init__(self,name,age,salary):
        self.name=name
        self.age=age
        self.salary=salary
 
print(People.__dict__)
p1=People('egon',18,3333.3)